In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like thi

In [3]:
#standard class syntax (Not pydantic)
# Doesn't do type checking
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email
        
foo = User(name="Joe",age="bar", email="joe@gmail.com")     
print(f"Joe's age is {foo.age}")  
foo
# we defined age as an INT, but it accepted a string happily.  WTH.

Joe's age is bar


In [4]:
# Pydantic allows you to strongly type your fields
class pUser(BaseModel):
    name: str
    age: int
    email: str
    
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")
foo_p #pydantic objects also print with useful information.

pUser(name='Jane', age=32, email='jane@gmail.com')

In [5]:
# This throws a validation error, because "bar" is not an INT.
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [7]:
# you can even next pydantic objects.  NOTE: a list of pUsers (defined above) is the data type for students.
class Class(BaseModel):
    students: List[pUser]

obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [ ]:
# this class is logically the same as the function that we defined in previous lessons
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")
    
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
weather_function = convert_pydantic_to_openai_function(WeatherSearch)